In [2]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Piauí - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [5]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_PI.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH Longevidade,Piauí - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Piauí - IDH Renda,Piauí - IDH,Piauí - Produção de Cimento (t),Piauí - Consumo de Cimento (t)
0,2014,2.597493e+07,1.458827e+06,6.481158,2.014649e+07,0.745344,0.652827,5.676413e+08,2.779340e+09,1.954361e+10,...,1002.930152,1312.189559,28.101214,20.242493,66.500996,1.263497,0.636024,0.676661,534.569599,755.713857
1,2015,2.623707e+07,1.443457e+06,6.481716,2.015777e+07,0.754836,0.669000,5.179392e+08,2.561731e+09,1.327596e+10,...,1002.890504,1261.987570,28.208348,20.043769,65.528148,1.276249,0.650948,0.690004,485.544522,794.945486
2,2016,2.645308e+07,1.433449e+06,6.489827,2.018916e+07,0.754094,0.671364,4.524144e+08,2.516859e+09,1.011370e+10,...,1002.832338,1235.205362,28.178052,20.095966,65.401854,1.267215,0.649398,0.690096,383.195013,842.687657
3,2017,2.658816e+07,1.433526e+06,6.502281,2.023043e+07,0.753012,0.674034,4.213689e+08,2.497710e+09,8.456309e+09,...,1002.920008,1214.156503,28.068432,20.444122,66.544636,1.228392,0.648888,0.690559,274.263754,831.356000
4,2018,2.657336e+07,1.435231e+06,6.498516,2.021580e+07,0.751558,0.674855,4.166580e+08,2.797145e+09,7.937760e+09,...,1002.983523,1225.305663,28.124040,20.588603,66.918659,1.220562,0.648515,0.690246,214.602396,626.223000
5,2019,2.636083e+07,1.440005e+06,6.475652,2.013490e+07,0.749700,0.675205,4.462625e+08,3.484252e+09,1.091282e+10,...,1002.894000,1274.660123,28.368985,20.453014,65.900441,1.237508,0.647652,0.689450,182.065812,702.437000
6,2020,2.598614e+07,1.444543e+06,6.443469,2.001866e+07,0.747400,0.673678,5.489412e+08,4.114911e+09,1.625333e+10,...,1002.778124,1246.811836,28.287959,20.572557,66.685432,1.202091,0.646803,0.687946,164.951348,788.164000
7,2021,2.538518e+07,1.435366e+06,6.400410,1.986583e+07,0.745527,0.671286,6.638811e+08,4.421848e+09,1.933031e+10,...,1002.720249,1192.333102,27.966206,20.640074,68.083428,1.160211,0.645694,0.686123,157.369748,826.368000
8,2022,2.466212e+07,1.424501e+06,6.348367,1.967997e+07,0.747302,0.668857,7.010332e+08,4.533216e+09,1.997888e+10,...,1002.715696,1195.956002,27.902157,20.532018,67.970748,1.160725,0.645393,0.685770,155.633436,804.417000


In [6]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH Longevidade,Piauí - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Piauí - IDH Renda,Piauí - IDH,Piauí - Produção de Cimento (t)
0,2.597493e+07,1.458827e+06,6.481158,2.014649e+07,0.745344,0.652827,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,18.366943,1002.930152,1312.189559,28.101214,20.242493,66.500996,1.263497,0.636024,0.676661,534.569599
1,2.623707e+07,1.443457e+06,6.481716,2.015777e+07,0.754836,0.669000,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,19.175926,1002.890504,1261.987570,28.208348,20.043769,65.528148,1.276249,0.650948,0.690004,485.544522
2,2.645308e+07,1.433449e+06,6.489827,2.018916e+07,0.754094,0.671364,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,22.137114,1002.832338,1235.205362,28.178052,20.095966,65.401854,1.267215,0.649398,0.690096,383.195013
3,2.658816e+07,1.433526e+06,6.502281,2.023043e+07,0.753012,0.674034,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,24.264986,1002.920008,1214.156503,28.068432,20.444122,66.544636,1.228392,0.648888,0.690559,274.263754
4,2.657336e+07,1.435231e+06,6.498516,2.021580e+07,0.751558,0.674855,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,...,22.693879,1002.983523,1225.305663,28.124040,20.588603,66.918659,1.220562,0.648515,0.690246,214.602396
5,2.636083e+07,1.440005e+06,6.475652,2.013490e+07,0.749700,0.675205,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,...,20.177485,1002.894000,1274.660123,28.368985,20.453014,65.900441,1.237508,0.647652,0.689450,182.065812
6,2.598614e+07,1.444543e+06,6.443469,2.001866e+07,0.747400,0.673678,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,...,23.289184,1002.778124,1246.811836,28.287959,20.572557,66.685432,1.202091,0.646803,0.687946,164.951348


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    794.945486
1    842.687657
2    831.356000
3    626.223000
4    702.437000
5    788.164000
6    826.368000
7    804.417000
8           NaN
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH Longevidade,Piauí - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Piauí - IDH Renda,Piauí - IDH,Piauí - Produção de Cimento (t)
0,-1.838455,2.041216,-0.730430,-0.918841,-1.902931,-2.148263,1.779974,0.019134,2.003355,-1.880579,...,-1.339334,0.474879,1.751573,-0.747937,-0.346127,0.658979,0.694390,-2.190406,-2.231167,1.422933
1,-0.601592,0.305771,-0.672436,-0.600943,1.067976,-0.070288,0.870373,-0.621387,0.401153,0.111653,...,-0.948054,-0.391520,0.242575,0.340320,-1.345391,-1.080549,1.301209,0.814083,0.432918,1.053568
2,0.417620,-0.824291,0.169810,0.283725,0.835686,0.233396,-0.328802,-0.753466,-0.407217,1.154682,...,0.484181,-1.662621,-0.562458,0.032575,-1.082924,-1.306373,0.871320,0.502002,0.451147,0.282444
3,1.054955,-0.815585,1.463065,1.446718,0.496919,0.576369,-0.896969,-0.809829,-0.830898,0.955301,...,1.513366,0.253206,-1.195156,-1.080931,0.667741,0.737012,-0.976024,0.399472,0.543637,-0.538268
4,0.985149,-0.623093,1.072105,1.034609,0.041995,0.681881,-0.983185,0.071542,-0.963455,0.100493,...,0.753471,1.641181,-0.860028,-0.516074,1.394248,1.405793,-1.348631,0.324223,0.481178,-0.987770
5,-0.017677,-0.084018,-1.302113,-1.245267,-0.539646,0.726905,-0.441390,2.094006,-0.202939,-0.441549,...,-0.463630,-0.315125,0.623494,1.972046,0.712452,-0.414861,-0.542265,0.150626,0.322287,-1.232907


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    794.945486
1    842.687657
2    831.356000
3    626.223000
4    702.437000
5    788.164000
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH Longevidade,Piauí - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Piauí - IDH Renda,Piauí - IDH,Piauí - Produção de Cimento (t)
6,-1.370251,0.391476,-2.128336,-2.114118,-1.052874,0.481774,1.16956,2.035192,0.985175,-0.705937,...,0.897153,-1.794404,-0.1971,0.975728,1.089265,0.857494,-1.577614,-0.018934,0.020272,-1.121037


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    826.368
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [12]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500, verbose=1)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(148, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(148, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(76, activation='tanh'),
        tf.keras.layers.Dense(36, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [13]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [14]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[139190908, 2326240897, 1137152011, 1101574145, 466168773, 3205770514, 2659015291, 46013031, 1570312540, 1600846411]


Step: 0 ___________________________________________


2023-08-20 10:34:14.359318: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-20 10:34:14.359732: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1603: early stopping
val_loss: 0.08962684869766235
winner_seed: 139190908


Step: 1 ___________________________________________
Epoch 1735: early stopping
val_loss: 0.0020731836557388306
winner_seed: 2326240897


Step: 2 ___________________________________________
Epoch 1629: early stopping
val_loss: 751.6077880859375


Step: 3 ___________________________________________
Epoch 1252: early stopping
val_loss: 2870.1904296875


Step: 4 ___________________________________________
Epoch 1213: early stopping
val_loss: 0.0002941526472568512
winner_seed: 466168773


Step: 5 ___________________________________________
Epoch 1337: early stopping
val_loss: 0.0012748874723911285


Step: 6 ___________________________________________
Epoch 1252: early stopping
val_loss: 0.10742582380771637


Step: 7 ___________________________________________
Epoch 1536: early stopping
val_loss: 0.009288046509027481


Step: 8 ___________________________________________
Epoch 1310: early stopping
val_loss: 74.9

In [15]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 0s 279ms/step - loss: 589345.4375 - val_loss: 636203.8750
Epoch 2/5000
1/1 [==============================] - 0s 12ms/step - loss: 568643.4375 - val_loss: 462536.3125
Epoch 3/5000
1/1 [==============================] - 0s 12ms/step - loss: 577954.2500 - val_loss: 761160.0000
Epoch 4/5000
1/1 [==============================] - 0s 12ms/step - loss: 517400.2812 - val_loss: 728793.6250
Epoch 5/5000
1/1 [==============================] - 0s 12ms/step - loss: 432770.3750 - val_loss: 517768.6250
Epoch 6/5000
1/1 [==============================] - 0s 12ms/step - loss: 506570.9062 - val_loss: 524399.6250
Epoch 7/5000
1/1 [==============================] - 0s 12ms/step - loss: 507996.3438 - val_loss: 539390.3750
Epoch 8/5000
1/1 [==============================] - 0s 12ms/step - loss: 511735.4062 - val_loss: 634201.0625
Epoch 9/5000
1/1 [==============================] - 0s 12ms/step - loss: 482118.6250 - val_loss: 637090.1250
Epoch 10/5000
1/1 

1/1 [==============================] - 0s 12ms/step - loss: 130148.4375 - val_loss: 222171.8281
Epoch 76/5000
1/1 [==============================] - 0s 12ms/step - loss: 108729.5234 - val_loss: 235458.7344
Epoch 77/5000
1/1 [==============================] - 0s 12ms/step - loss: 140739.8281 - val_loss: 237095.4688
Epoch 78/5000
1/1 [==============================] - 0s 16ms/step - loss: 91149.5391 - val_loss: 256583.8594
Epoch 79/5000
1/1 [==============================] - 0s 12ms/step - loss: 133826.4219 - val_loss: 228128.5312
Epoch 80/5000
1/1 [==============================] - 0s 12ms/step - loss: 116660.7891 - val_loss: 210587.9688
Epoch 81/5000
1/1 [==============================] - 0s 12ms/step - loss: 135135.7344 - val_loss: 237046.3750
Epoch 82/5000
1/1 [==============================] - 0s 12ms/step - loss: 118291.7500 - val_loss: 244205.2344
Epoch 83/5000
1/1 [==============================] - 0s 12ms/step - loss: 104512.4375 - val_loss: 230109.9375
Epoch 84/5000
1/1 [======

Epoch 150/5000
1/1 [==============================] - 0s 16ms/step - loss: 45518.3477 - val_loss: 76969.2266
Epoch 151/5000
1/1 [==============================] - 0s 16ms/step - loss: 36758.6836 - val_loss: 78949.9219
Epoch 152/5000
1/1 [==============================] - 0s 12ms/step - loss: 41456.9023 - val_loss: 77061.1719
Epoch 153/5000
1/1 [==============================] - 0s 12ms/step - loss: 48649.3242 - val_loss: 77048.2266
Epoch 154/5000
1/1 [==============================] - 0s 12ms/step - loss: 34552.4258 - val_loss: 76168.8203
Epoch 155/5000
1/1 [==============================] - 0s 12ms/step - loss: 37671.1172 - val_loss: 73395.5234
Epoch 156/5000
1/1 [==============================] - 0s 13ms/step - loss: 40995.8984 - val_loss: 72332.7266
Epoch 157/5000
1/1 [==============================] - 0s 13ms/step - loss: 33730.4961 - val_loss: 71897.5312
Epoch 158/5000
1/1 [==============================] - 0s 13ms/step - loss: 40367.2109 - val_loss: 71754.8906
Epoch 159/5000
1/1 

1/1 [==============================] - 0s 12ms/step - loss: 9822.7188 - val_loss: 32848.2227
Epoch 226/5000
1/1 [==============================] - 0s 12ms/step - loss: 11557.1143 - val_loss: 31205.4043
Epoch 227/5000
1/1 [==============================] - 0s 12ms/step - loss: 13256.4150 - val_loss: 32019.6777
Epoch 228/5000
1/1 [==============================] - 0s 12ms/step - loss: 13259.0459 - val_loss: 32108.1172
Epoch 229/5000
1/1 [==============================] - 0s 12ms/step - loss: 12587.8154 - val_loss: 31593.3477
Epoch 230/5000
1/1 [==============================] - 0s 16ms/step - loss: 8410.3818 - val_loss: 29791.0254
Epoch 231/5000
1/1 [==============================] - 0s 16ms/step - loss: 12632.5000 - val_loss: 30051.3574
Epoch 232/5000
1/1 [==============================] - 0s 14ms/step - loss: 8615.5713 - val_loss: 29474.2285
Epoch 233/5000
1/1 [==============================] - 0s 12ms/step - loss: 10138.6172 - val_loss: 28974.2559
Epoch 234/5000
1/1 [=================

Epoch 301/5000
1/1 [==============================] - 0s 12ms/step - loss: 8411.7295 - val_loss: 18780.8926
Epoch 302/5000
1/1 [==============================] - 0s 12ms/step - loss: 4400.0298 - val_loss: 19964.5820
Epoch 303/5000
1/1 [==============================] - 0s 14ms/step - loss: 4384.7007 - val_loss: 16741.0332
Epoch 304/5000
1/1 [==============================] - 0s 14ms/step - loss: 1419.8179 - val_loss: 17220.2344
Epoch 305/5000
1/1 [==============================] - 0s 12ms/step - loss: 4337.5811 - val_loss: 17437.0859
Epoch 306/5000
1/1 [==============================] - 0s 12ms/step - loss: 9118.0283 - val_loss: 16220.3018
Epoch 307/5000
1/1 [==============================] - 0s 12ms/step - loss: 2570.1687 - val_loss: 16484.5449
Epoch 308/5000
1/1 [==============================] - 0s 12ms/step - loss: 1806.9061 - val_loss: 17343.2852
Epoch 309/5000
1/1 [==============================] - 0s 12ms/step - loss: 761.9380 - val_loss: 17169.6035
Epoch 310/5000
1/1 [=========

...

1/1 [==============================] - 0s 13ms/step - loss: 1384.1281 - val_loss: 148.7718
Epoch 841/5000
1/1 [==============================] - 0s 12ms/step - loss: 937.2681 - val_loss: 192.0011
Epoch 842/5000
1/1 [==============================] - 0s 12ms/step - loss: 790.0236 - val_loss: 227.1474
Epoch 843/5000
1/1 [==============================] - 0s 12ms/step - loss: 732.7298 - val_loss: 165.2380
Epoch 844/5000
1/1 [==============================] - 0s 12ms/step - loss: 915.1298 - val_loss: 179.2091
Epoch 845/5000
1/1 [==============================] - 0s 12ms/step - loss: 864.5893 - val_loss: 241.3847
Epoch 846/5000
1/1 [==============================] - 0s 12ms/step - loss: 672.0221 - val_loss: 173.7943
Epoch 847/5000
1/1 [==============================] - 0s 12ms/step - loss: 897.6517 - val_loss: 87.4688
Epoch 848/5000
1/1 [==============================] - 0s 12ms/step - loss: 1366.8154 - val_loss: 37.3864
Epoch 849/5000
1/1 [==============================] - 0s 12ms/step - l

1/1 [==============================] - 0s 12ms/step - loss: 876.3735 - val_loss: 115.2596
Epoch 919/5000
1/1 [==============================] - 0s 12ms/step - loss: 1196.5232 - val_loss: 117.2668
Epoch 920/5000
1/1 [==============================] - 0s 12ms/step - loss: 638.1949 - val_loss: 59.5772
Epoch 921/5000
1/1 [==============================] - 0s 12ms/step - loss: 868.4670 - val_loss: 20.8014
Epoch 922/5000
1/1 [==============================] - 0s 12ms/step - loss: 911.3274 - val_loss: 45.2951
Epoch 923/5000
1/1 [==============================] - 0s 12ms/step - loss: 222.1116 - val_loss: 49.0120
Epoch 924/5000
1/1 [==============================] - 0s 12ms/step - loss: 584.9600 - val_loss: 79.2094
Epoch 925/5000
1/1 [==============================] - 0s 12ms/step - loss: 2420.4106 - val_loss: 96.4421
Epoch 926/5000
1/1 [==============================] - 0s 12ms/step - loss: 2276.9141 - val_loss: 2.3860
Epoch 927/5000
1/1 [==============================] - 0s 12ms/step - loss: 

1/1 [==============================] - 0s 12ms/step - loss: 824.7327 - val_loss: 604.2823
Epoch 997/5000
1/1 [==============================] - 0s 12ms/step - loss: 405.9695 - val_loss: 641.0255
Epoch 998/5000
1/1 [==============================] - 0s 12ms/step - loss: 1485.2690 - val_loss: 551.5674
Epoch 999/5000
1/1 [==============================] - 0s 12ms/step - loss: 1452.8309 - val_loss: 450.8520
Epoch 1000/5000
1/1 [==============================] - 0s 11ms/step - loss: 1003.6815 - val_loss: 646.8522
Epoch 1001/5000
1/1 [==============================] - 0s 12ms/step - loss: 398.9391 - val_loss: 667.1772
Epoch 1002/5000
1/1 [==============================] - 0s 13ms/step - loss: 710.8653 - val_loss: 683.4918
Epoch 1003/5000
1/1 [==============================] - 0s 12ms/step - loss: 1060.6178 - val_loss: 748.2649
Epoch 1004/5000
1/1 [==============================] - 0s 12ms/step - loss: 936.4404 - val_loss: 714.9030
Epoch 1005/5000
1/1 [==============================] - 0s 12m

1/1 [==============================] - 0s 12ms/step - loss: 1278.0283 - val_loss: 765.3307
Epoch 1073/5000
1/1 [==============================] - 0s 12ms/step - loss: 614.6426 - val_loss: 844.8453
Epoch 1074/5000
1/1 [==============================] - 0s 11ms/step - loss: 2544.8816 - val_loss: 714.4266
Epoch 1075/5000
1/1 [==============================] - 0s 11ms/step - loss: 781.5698 - val_loss: 852.4696
Epoch 1076/5000
1/1 [==============================] - 0s 12ms/step - loss: 1244.4449 - val_loss: 813.2940
Epoch 1077/5000
1/1 [==============================] - 0s 13ms/step - loss: 787.0522 - val_loss: 700.6012
Epoch 1078/5000
1/1 [==============================] - 0s 13ms/step - loss: 1007.1533 - val_loss: 989.2377
Epoch 1079/5000
1/1 [==============================] - 0s 12ms/step - loss: 576.2938 - val_loss: 1072.0309
Epoch 1080/5000
1/1 [==============================] - 0s 12ms/step - loss: 1376.3584 - val_loss: 1326.5262
Epoch 1081/5000
1/1 [==============================] - 

1/1 [==============================] - 0s 11ms/step - loss: 2528.0735 - val_loss: 49.6275
Epoch 1150/5000
1/1 [==============================] - 0s 12ms/step - loss: 1544.3268 - val_loss: 1.2641
Epoch 1151/5000
1/1 [==============================] - 0s 11ms/step - loss: 477.4636 - val_loss: 0.8540
Epoch 1152/5000
1/1 [==============================] - 0s 11ms/step - loss: 463.6070 - val_loss: 2.0224
Epoch 1153/5000
1/1 [==============================] - 0s 11ms/step - loss: 1193.3730 - val_loss: 0.1100
Epoch 1154/5000
1/1 [==============================] - 0s 11ms/step - loss: 467.9930 - val_loss: 0.1013
Epoch 1155/5000
1/1 [==============================] - 0s 11ms/step - loss: 131.9178 - val_loss: 583.5952
Epoch 1156/5000
1/1 [==============================] - 0s 11ms/step - loss: 600.3753 - val_loss: 453.2736
Epoch 1157/5000
1/1 [==============================] - 0s 11ms/step - loss: 153.7675 - val_loss: 479.9533
Epoch 1158/5000
1/1 [==============================] - 0s 11ms/step - 

In [16]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 45ms/step
[826.368](test_target) - [[755.8802]](prediction) = 70.4878120117188


In [17]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [18]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH Longevidade,Piauí - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Piauí - IDH Renda,Piauí - IDH,Piauí - Produção de Cimento (t)
0,-1.262173,1.301349,-0.77642,-0.996119,-1.410715,-1.404478,1.165029,1.396038,1.335266,-1.330895,...,-0.941860,1.140654,1.328603,-1.359974,1.368086,1.406397,-1.025286,-1.407890,-1.414189,1.059247
1,0.078648,-0.171229,-0.63545,-0.371313,0.791449,0.558779,0.111769,-0.502296,-0.264151,0.251276,...,-0.442675,0.153676,-0.244646,1.015923,-0.994295,-0.574612,1.356202,0.819633,0.699862,0.281855
2,1.183526,-1.130120,1.41187,1.367433,0.619266,0.845699,-1.276798,-0.893743,-1.071116,1.079619,...,1.384535,-1.294330,-1.083957,0.344051,-0.373790,-0.831786,-0.330915,0.588257,0.714327,-1.341102


train_target:


,Piauí - Consumo de Cimento (t)
0,794.945486
1,842.687657
2,831.356000


test_input:


...

,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH Longevidade,Piauí - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Piauí - IDH Renda,Piauí - IDH,Piauí - Produção de Cimento (t)
6,-1.370251,0.391476,-2.128336,-2.114118,-1.052874,0.481774,1.16956,2.035192,0.985175,-0.705937,...,0.897153,-1.794404,-0.1971,0.975728,1.089265,0.857494,-1.577614,-0.018934,0.020272,-1.121037


test_target:


,Piauí - Consumo de Cimento (t)
6,826.368


Epoch 1213: early stopping
1/1 [==============================] - 0s 30ms/step
Error: 70.4878120117188




[836.7271118164062, 694.71484375, 668.73681640625, 755.8801879882812]

In [19]:
display(targets)
display(predictions)

[626.223, 702.437, 788.164, 826.368]

[836.7271118164062, 694.71484375, 668.73681640625, 755.8801879882812]

In [20]:
mae = mean_absolute_error(predictions, targets)
mae

102.03531591796877

In [21]:
porcentage = mae/np.mean(targets)
porcentage

0.13867299981512424